EXERCISE 3.2

In [70]:
import numpy as np 
import scipy.linalg as lin
import scipy.sparse as sp
import matplotlib.pyplot as plt

In [85]:
# radius is set tot 2

# n = 41 makes the spacing nice; n = 21 is another options.
# In general, the spacing has to be chosen s.t. the source coordinate can be recognized.
#  

def laplacian_circle_matrix(n): 
    N = n * n  
    laplacian = np.zeros((N, N)) 
    points_within_boundary = []
    
    x = np.linspace(-2, 2, n)
    y = np.linspace(-2, 2, n)
    

    # This as all the same as in 3.1:
    for i in range(n):
        for j in range(n):
            if x[i]**2 + y[j]**2 <= 4:
                index = i * n + j  # corresponding index in the stacked vector
                points_within_boundary.append(index)
    
    for i in range(N):  
        if i in points_within_boundary:  
            laplacian[i, i] = -4  

        if (i - 1 in points_within_boundary) and (i % n) != 0:
            laplacian[i, i - 1] = 1
            laplacian[i - 1, i] = 1 

        if (i + 1 in points_within_boundary) and ((i + 1) % n) != 0:
            laplacian[i, i + 1] = 1
            laplacian[i + 1, i] = 1  

        if (i - n in points_within_boundary):
            laplacian[i, i - n] = 1
            laplacian[i - n, i] = 1  

        if (i + n in points_within_boundary):
            laplacian[i, i + n] = 1
            laplacian[i + n, i] = 1

    # We also need to incorporate the source, this means overwriting 1 row in the matrix:
    index_source_row =  np.where(np.isclose(x,0.6))[0]*n + np.where(np.isclose(y,1.2))[0] #There really should be a way to make this cleaner
    laplacian[index_source_row,:] = 0
    laplacian[index_source_row,index_source_row] = 1

    return points_within_boundary


In [73]:
# Let M equal to matrix returned by the function above. b = (0,0,...,0,1,0,...,0,0), more carefully defined in the report.
# We now solve Mc = b
# We first consider the system for n = 21

x = np.linspace(-2, 2, 21)
y = np.linspace(-2, 2, 21)

b = np.zeros(441)
b[np.where(np.isclose(x,0.6))[0]*21 + np.where(np.isclose(y,1.2))[0]] = 1

M = laplacian_circle_matrix(21)

In [74]:
lin.solve(M,b)

LinAlgError: Matrix is singular.

===================================================================================================================================================

HERE, WE TRY TO SOLVE THE PROBLEM USING METHOD ONE OF THE HINT

In [ ]:
#DEZE IS VOLGENS MIJ GOED!!!!!!!!!!!!!!!!!!

def laplacian_circle_fix(n, L):
    R = L/2  
    coordinates_in_circle = [] 
    dictionary_storing = {} 

    x = np.linspace(-R, R, n)
    y = np.linspace(-R, R, n)

    count = 0 
    for i in range(n):
        for j in range(n):
            if x[i]**2 + y[j]**2 <= R**2:
                coordinates_in_circle.append((i, j))
                dictionary_storing[(i, j)] = count
                count += 1

    dim = len(coordinates_in_circle) 
    matrix_M = np.zeros((dim, dim)) 

    for count, (i, j) in enumerate(coordinates_in_circle):
        matrix_M[count, count] = -4 
        for a, b in [(-1, 0), (1, 0), (0, -1), (0, 1)]: 
            aa, bb = i + a, j + b
            if (aa, bb) in dictionary_storing: 
                neighbor_index = dictionary_storing[(aa, bb)]
                matrix_M[count, neighbor_index] = 1

    return matrix_M

In [99]:
laplacian_circle_fix(5,1)

array([[-4.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -4.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1., -4.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1., -4.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -4.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  1., -4.,  1.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  1., -4.,  1.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  1., -4.,  1.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -4.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -4.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1., -4.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1., -4.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -4.]])